Write a program to generate Hash code for a given text using SHA 512

In [1]:
import cryptography
import hashlib

In [2]:
def sha512(message):
    hash_object = hashlib.sha512(message.encode())
    hex_dig = hash_object.hexdigest()
    return hex_dig

In [10]:
msg = input("Enter your message:")
print(sha512(msg))

Enter your message:Hello World
2c74fd17edafd80e8447b0d46741ee243b7eb74dd2149a0ab1b9246fb30382f27e853d8585719e0e67cbda0daa8f51671064615d645ae27acb15bfb1447f459b


Write a program to send a message ensuring confidentiality and integrity
Hint for 2:
perform diffie hellman exchange (from previous lab) to generate a session key
Compute Hash code of the message text
use AES to encrypt the (text + hash code)
encrypt the (message + Hash Code)
send it to client/server
decrypt the message
check the integrity.

In [4]:
from Crypto.Cipher import AES 
from Crypto.Util.Padding import pad, unpad
import socket
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

def format_key(key):
    return key.ljust(16)[:16].encode()

def encrypt(message, key):
    key = format_key(key)  
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return cipher.encrypt(pad(message.encode(), AES.block_size))

def decrypt(ciphertext, key):
    key = format_key(key)
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ciphertext), AES.block_size).decode()

In [9]:
import socket
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 1234))
    server_socket.listen(1)
    print("Server listening on port 12345...")
    
    conn, addr = server_socket.accept()
    print(f"Connection established with {addr}")
    
    server_private_key = ec.generate_private_key(ec.SECP256R1())
    server_public_key = server_private_key.public_key()

    server_public_bytes = server_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    conn.sendall(server_public_bytes)

    client_public_bytes = conn.recv(1024)
    client_public_key = serialization.load_pem_public_key(client_public_bytes)

    shared_secret = server_private_key.exchange(ec.ECDH(), client_public_key)
    print("Server derived shared secret:", shared_secret.hex())
    
    msg = input("Enter a message to send:")
    msg_hash = sha512(msg)
    
    text_to_send = msg+ "***" +msg_hash
    print(text_to_send)
    
    encrypt_msg = encrypt(text_to_send,shared_secret.hex())
    conn.send(encrypt_msg)
    
    
    
    
    conn.close()
    server_socket.close()

server()

Server listening on port 12345...
Connection established with ('127.0.0.1', 52432)
Server derived shared secret: 2ea18cea6d88fc29cf3ed9cc6fb63addbeaba31ce5fbb4c5cb7a5ea0f05ac0c1
Enter a message to send:hi
hi***150a14ed5bea6cc731cf86c41566ac427a8db48ef1b9fd626664b3bfbb99071fa4c922f33dde38719b8c8354e2b7ab9d77e0e67fc12843920a712e73d558e197
